Extraindo informações do Bureau Balance: 
* https://www.kaggle.com/c/home-credit-default-risk/discussion/58445
* https://www.kaggle.com/c/home-credit-default-risk/discussion/57750

In [1]:
import pandas as pd
import numpy as np
import data
import charts
import pickle
import seaborn as sns

pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
def find_missing(data):
    # number of missing values
    count_missing = data.isnull().sum().values
    # total records
    total = data.shape[0]
    # percentage of missing
    ratio_missing = count_missing / total
    # return a dataframe to show: feature name, # of missing and % of missing
    return pd.DataFrame(data={'missing_count':count_missing, 'missing_ratio':ratio_missing}, index=data.columns.values)

In [19]:
application_train_df = pd.read_csv('dataset/application_train.csv')
application_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Bureau features

In [21]:
# Montagem da tabela relacionada ao bureau
bureau_full_df = pd.read_csv('dataset/bureau.csv')
bureau_balance_df = pd.read_csv('dataset/bureau_balance.csv')
#bureau_full_df = bureau_df.join(bureau_balance_df.set_index('SK_ID_BUREAU'), on='SK_ID_BUREAU')

# Adicionando o target para facilitar analises futuras e treinamento do meu futuro modelo
#bureau_full_df = bureau_full_df.join(application_train_df.set_index('SK_ID_CURR')['TARGET'], on='SK_ID_CURR')
bureau_balance_df.head(5)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [14]:
bureau_full_df.columns

Index(['SK_ID_CURR', 'BUREAU_LOAN_COUNT', 'BUREAU_LOAN_TYPES',
       'AVERAGE_LOAN_TYPE', 'ACTIVE_LOANS_PERCENTAGE', 'MIN_DAYS_DIFF',
       'MAX_DAYS_DIFF', 'MEAN_DAYS_DIFF', 'STD_DAYS_DIFF',
       'CREDIT_ENDDATE_PERCENTAGE', 'MIN_DAYS_ENDDATE_DIFF',
       'MAX_DAYS_ENDDATE_DIFF', 'MEAN_DAYS_ENDDATE_DIFF',
       'STD_DAYS_ENDDATE_DIFF', 'AVG_ENDDATE_FUTURE', 'TOTAL_CUSTOMER_DEBT',
       'TOTAL_CUSTOMER_CREDIT', 'DEBT_CREDIT_RATIO', 'TOTAL_CUSTOMER_OVERDUE',
       'OVERDUE_DEBT_RATIO', 'AVG_CREDITDAYS_PROLONGED'],
      dtype='object')

In [12]:
bureau_full_df[bureau_full_df.TOTAL_CUSTOMER_OVERDUE > 0]

,SK_ID_CURR,SK_ID_BUREAU,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF,CREDIT_ENDDATE_PERCENTAGE,DAYS_ENDDATE_DIFF,AVG_ENDDATE_FUTURE,TOTAL_CUSTOMER_DEBT,TOTAL_CUSTOMER_CREDIT,DEBT_CREDIT_RATIO,TOTAL_CUSTOMER_OVERDUE,OVERDUE_DEBT_RATIO,AVG_CREDITDAYS_PROLONGED
365,282297,5714943,11,3,3.666667,0.454545,98,0.272727,NaN,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
366,282297,5714944,11,3,3.666667,0.454545,370,0.272727,NaN,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
367,282297,5714945,11,3,3.666667,0.454545,118,0.272727,NaN,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
368,282297,5714947,11,3,3.666667,0.454545,261,0.272727,313.0,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
369,282297,5714948,11,3,3.666667,0.454545,109,0.272727,257.0,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
370,282297,5714952,11,3,3.666667,0.454545,817,0.272727,NaN,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
371,282297,5714953,11,3,3.666667,0.454545,273,0.272727,NaN,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
372,282297,5714954,11,3,3.666667,0.454545,0,0.272727,0.0,190.000000,6.294134e+05,3.693748e+06,0.170400,231.525,3.678425e-04,0.000000
782,408647,5715465,5,1,5.000000,0.600000,160,0.600000,NaN,344.000000,1.088064e+06,2.495084e+06,0.436083,288.000,2.646903e-04,0.000000
783,408647,5715466,5,1,5.000000,0.600000,1042,0.600000,NaN,344.000000,1.088064e+06,2.495084e+06,0.436083,288.000,2.646903e-04,0.000000


In [10]:
df = bureau_full_df
#grp = df.groupby(by=['SK_ID_CURR'])['DAYS_ENDDATE_DIFF'].mean().reset_index().rename(index=str, columns={
#    feature_name: 'AVERAGE_%s' % feature_name.upper()})

agg_func = ['min', 'max', 'mean']
feature_name = 'DAYS_ENDDATE_DIFF'

columns = {}
for f in agg_func:
    columns[f] = f.upper() + '_' + feature_name

grp = df.groupby(by=['SK_ID_CURR'])[feature_name].agg(agg_func)
grp.reset_index().rename(index=str, columns=columns)

,SK_ID_CURR,MIN_DAYS_ENDDATE_DIFF,MAX_DAYS_ENDDATE_DIFF,MEAN_DAYS_ENDDATE_DIFF
0,100001,0.0,876.0,455.666667
1,100002,0.0,695.0,143.600000
2,100003,0.0,0.0,0.000000
3,100004,NaN,NaN,NaN
4,100005,0.0,1202.0,601.000000
5,100007,NaN,NaN,NaN
6,100008,0.0,0.0,0.000000
7,100009,0.0,905.0,252.600000
8,100010,0.0,0.0,0.000000
9,100011,NaN,NaN,NaN


In [39]:
bureau_full_df['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan'], dtype=object)

In [18]:
with open('dataset/bureau_full.pkl', 'wb') as handle:
    model_predicts = pickle.dump(bureau_full_df, handle)

### NOTAS
> * Numeros negativos em datas (Ex.: DAYS_CREDIT, DAYS_CREDIT_ENDDATE etc.) significam que e no passado e positivos no futuro

### TODO:
> Ao verificar o mesmo cliente, podemos identificar que o mesmo pode estar registrado em diversos "escritórios" diferentes. 
> Isso pode gerar uma série de features interessantes:
> * OK: Quantos créditos já foram pedidos por este cliente?
> * OK: Gerar features baseados no numero de itens para cada cliente no CREDIT_ACTIVE
> * OK: Qual o tempo entre um emprestimo e outro?
> * OK: Qual o tempo médio entre um emprestimo e outro?
> * OK: Numero total de dias do emprestimo e tempo médio de emprestimos realizados
> * Existem dois ou mais pedidos de empréstimo ao mesmo tempo?
> * Existem créditos atrasados (CREDIT_DAY_OVERDUE)?
>   * Quantos creditos estao atrasados?
> * Quantos creditos ativos e nao ativos de cada tipo (CREDIT_TYPE) foram abertos?
> * Qual a porcentagem ja foi paga do credito (DAYS_CREDIT_ENDDATE e DAYS_CREDIT depois normalizar)
> * Qual a taxa de inadimplencia por SK_ID_BUREAU?

#### Quantos créditos já foram pedidos por este cliente?

In [128]:
bureau_full_df[(bureau_full_df.CREDIT_ACTIVE == 'Closed') & (bureau_full_df.DAYS_CREDIT_ENDDATE > 0)]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN
353,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,0,139.285714
354,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,343,139.285714
355,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,28,139.285714
356,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,131,139.285714
357,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,299,139.285714
358,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,123,139.285714
359,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,51,139.285714
360,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,0,139.285714
361,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,343,139.285714
362,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,28,139.285714


In [80]:
x = bureau_full_df[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby(by = ['SK_ID_CURR'])
x = x['SK_ID_BUREAU'].count().reset_index()
x = x.rename(index=str, columns={'SK_ID_BUREAU': 'BUREAU_NUMBER_OF_LOANS'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')

In [81]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5,7
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5,7
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5,7


#### Gerar features baseadas no número de itens para cada cliente no CREDIT_ACTIVE

In [82]:
for credit_active_state in bureau_full_df['CREDIT_ACTIVE'].unique():
    x = bureau_full_df[bureau_full_df.CREDIT_ACTIVE == credit_active_state][['SK_ID_CURR', 'CREDIT_ACTIVE']].groupby(by = ['SK_ID_CURR'])
    x = x['CREDIT_ACTIVE'].count().reset_index()

    new_column_name = 'BUREAU_NUMBER_OF_%s_LOANS' % credit_active_state.upper().replace(' ', '_')
    x = x.rename(index=str, columns={'CREDIT_ACTIVE': new_column_name})
    bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')
    bureau_full_df[new_column_name] = bureau_full_df[new_column_name].fillna(0)

In [83]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5,7,4.0,3.0,0.0,0.0
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5,7,4.0,3.0,0.0,0.0
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5,7,4.0,3.0,0.0,0.0


#### Qual o tempo entre um emprestimo e outro?

In [84]:
grp = bureau_full_df[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT'] * -1
grp1['DAYS_CREDIT_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_CREDIT_DIFF'] = grp1['DAYS_CREDIT_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT'], grp1['DAYS_CREDIT1']
grp1.head()

bureau_full_df = bureau_full_df.merge(grp1, on = ['SK_ID_CURR'], how = 'left')

In [85]:
# Qual o tempo medio entre os emprestimos realizados?
x = bureau_full_df[['SK_ID_CURR', 'DAYS_CREDIT_DIFF']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_DIFF'].mean()
x = x.to_frame().reset_index().rename(index=str, columns={'DAYS_CREDIT_DIFF': 'DAYS_CREDIT_DIFF_MEAN'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')

In [86]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100002]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN
6169431,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,0,166.75
6169432,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,373,166.75
6169433,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,169,166.75
6169434,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,397,166.75
6169435,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,1,166.75
6169436,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,78,166.75
6169437,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,4,166.75
6169438,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,312,166.75
6169439,100002,6158905,Closed,currency 1,-476,0,NaN,-48.0,NaN,0,0.000,0.0,NaN,0.0,Credit card,-47,NaN,8,6.0,2.0,0.0,0.0,0,166.75
6169440,100002,6158905,Closed,currency 1,-476,0,NaN,-48.0,NaN,0,0.000,0.0,NaN,0.0,Credit card,-47,NaN,8,6.0,2.0,0.0,0.0,373,166.75


#### Numero total de dias do emprestimo

In [87]:
x = bureau_full_df
x['DAYS_CREDIT_ENDDATE'] = x['DAYS_CREDIT_ENDDATE'].fillna(0)
x['DAYS_CREDIT_POSITIVE'] = x['DAYS_CREDIT'] * -1
bureau_full_df['TOTAL_OF_LOAN_DAYS'] = x['DAYS_CREDIT_POSITIVE'] + x['DAYS_CREDIT_ENDDATE']
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,160,166.272727,497,344.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,5,166.272727,497,344.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,65,166.272727,497,344.0


In [88]:
# Qual o tempo medio entre os emprestimos realizados?
x = bureau_full_df[['SK_ID_CURR', 'TOTAL_OF_LOAN_DAYS']].groupby(by = ['SK_ID_CURR'])['TOTAL_OF_LOAN_DAYS'].mean()
x = x.to_frame().reset_index().rename(index=str, columns={'TOTAL_OF_LOAN_DAYS': 'MEAN_OF_LOAN_DAYS'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0,3169.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,160,166.272727,497,344.0,3169.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0,3169.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,5,166.272727,497,344.0,3169.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,65,166.272727,497,344.0,3169.0


#### Existem dois ou mais pedidos de empréstimo ao mesmo tempo?

#### Existem créditos atrasados (CREDIT_DAY_OVERDUE)?

#### Gravando nova tabela em CSV para ser usada no modelo externo

In [89]:
application_train_df = pd.read_csv('dataset/application_train.csv')

In [90]:
features = ['SK_ID_CURR',  'BUREAU_NUMBER_OF_LOANS', 'BUREAU_NUMBER_OF_CLOSED_LOANS', 'BUREAU_NUMBER_OF_ACTIVE_LOANS', 'BUREAU_NUMBER_OF_SOLD_LOANS', 'DAYS_CREDIT_DIFF_MEAN', 'MEAN_OF_LOAN_DAYS']
b = bureau_full_df[features].drop_duplicates().reset_index()
b.to_csv('dataset/bureau_processed.csv', index=False)

In [92]:
x = application_train_df.merge(right=b, on=['SK_ID_CURR'], how='left')
x.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,index,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,DAYS_CREDIT_DIFF_MEAN,MEAN_OF_LOAN_DAYS
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,6169431.0,8.0,6.0,2.0,0.0,166.75,612.25
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9351490.0,4.0,3.0,1.0,0.0,495.00,856.25
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-

In [93]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100006]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS


In [59]:
application_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [55]:
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,0,83.714286,497,344.0,4606.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,160,83.714286,497,344.0,4606.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,0,83.714286,497,344.0,4606.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,5,83.714286,497,344.0,4606.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,65,83.714286,497,344.0,4606.0


In [18]:
features = ['SK_ID_CURR', 'BUREAU_NUMBER_OF_LOANS', 'BUREAU_NUMBER_OF_CLOSED_LOANS', 'BUREAU_NUMBER_OF_ACTIVE_LOANS', 'BUREAU_NUMBER_OF_SOLD_LOANS', 'BUREAU_NUMBER_OF_BAD DEBT_LOANS', 'DAYS_CREDIT_DIFF', 'DAYS_CREDIT_DIFF_MEAN', 'MEAN_OF_LOAN_DAYS']
bureau_full_df[features].to_csv('dataset/bureau_processed.csv', index=False)

### Analisando bureau_balance.csv

In [131]:
from sklearn.preprocessing import MinMaxScaler

bureau_balance_df = pd.read_csv('dataset/bureau_balance.csv')
bureau_balance_df['MONTHS_BALANCE'] = bureau_balance_df['MONTHS_BALANCE'] * -1
bureau_balance_df['STATUS_NUMBER'] = pd.to_numeric(bureau_balance_df['STATUS'].map({'C': 0,   # Pagamento feito normal
                                                                                    'X': 0.1, # Nao tem a informacao, vamos dar um risco baixo
                                                                                    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}))
max_months = bureau_balance_df['MONTHS_BALANCE'].max()
bureau_balance_df['RISK_SCORE'] = ((max_months - bureau_balance_df['MONTHS_BALANCE']) * bureau_balance_df['STATUS_NUMBER'])
bureau_balance_df['RISK_SCORE'] = bureau_balance_df['RISK_SCORE'] / bureau_balance_df['RISK_SCORE'].max()

In [132]:
bureau_balance_df['RISK_SCORE'].max()

1.0

In [133]:
bureau_balance_df.head(10)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,STATUS_NUMBER,RISK_SCORE
0,5715448,0,C,0.0,0.0
1,5715448,1,C,0.0,0.0
2,5715448,2,C,0.0,0.0
3,5715448,3,C,0.0,0.0
4,5715448,4,C,0.0,0.0
5,5715448,5,C,0.0,0.0
6,5715448,6,C,0.0,0.0
7,5715448,7,C,0.0,0.0
8,5715448,8,C,0.0,0.0
9,5715448,9,0,0.0,0.0


In [134]:
# Alguns casos com informações faltantes mas nada demais
print(bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 5715448]['RISK_SCORE'].sum())
bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 5715448]

0.158958333333


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,STATUS_NUMBER,RISK_SCORE
0,5715448,0,C,0.0,0.000000
1,5715448,1,C,0.0,0.000000
2,5715448,2,C,0.0,0.000000
3,5715448,3,C,0.0,0.000000
4,5715448,4,C,0.0,0.000000
5,5715448,5,C,0.0,0.000000
6,5715448,6,C,0.0,0.000000
7,5715448,7,C,0.0,0.000000
8,5715448,8,C,0.0,0.000000
9,5715448,9,0,0.0,0.000000


In [135]:
# Risco altíssimo, os ultimos meses todos com status 5
print(bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 5724857]['RISK_SCORE'].sum())
bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 5724857]

5.84375


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,STATUS_NUMBER,RISK_SCORE
73909,5724857,0,5,5.0,1.000000
73910,5724857,1,5,5.0,0.989583
73911,5724857,2,5,5.0,0.979167
73912,5724857,3,5,5.0,0.968750
73913,5724857,4,5,5.0,0.958333
73914,5724857,5,5,5.0,0.947917
73915,5724857,6,0,0.0,0.000000
73916,5724857,7,0,0.0,0.000000
73917,5724857,8,0,0.0,0.000000
73918,5724857,9,0,0.0,0.000000


In [129]:
# Ultimos 4 meses mais recentes atrasados
print(bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 6570709]['RISK_SCORE'].sum())
bureau_balance_df[bureau_balance_df.SK_ID_BUREAU == 6570709]

0.0105229160579


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,STATUS_NUMBER,RISK_SCORE
887378,6570709,0,4,4.0,1.041667e-02
887379,6570709,1,3,3.0,1.051513e-04
887380,6570709,2,2,2.0,1.083676e-06
887381,6570709,3,1,1.0,1.140583e-08
887382,6570709,4,0,0.0,1.226433e-10
887383,6570709,5,0,0.0,1.226433e-10
887384,6570709,6,0,0.0,1.226433e-10
887385,6570709,7,0,0.0,1.226433e-10
887386,6570709,8,0,0.0,1.226433e-10
887387,6570709,9,0,0.0,1.226433e-10


In [136]:
bureau_balance_df.groupby(by='SK_ID_BUREAU')['RISK_SCORE'].sum().reset_index()

,SK_ID_BUREAU,RISK_SCORE
0,5001709,0.011458
1,5001710,0.178958
2,5001711,0.020000
3,5001712,0.000000
4,5001713,0.391875
5,5001714,0.278125
6,5001715,0.831250
7,5001716,0.085417
8,5001717,0.000000
9,5001718,0.517083


In [82]:
bureau_balance_df[bureau_balance_df.STATUS == '4'][bureau_balance_df.MONTHS_BALANCE == 0]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,STATUS_NUMBER,TRUST_SCORE
887378,6570709,0,4,4.0,1.0
1774273,6596848,0,4,4.0,1.0
1858554,6007250,0,4,4.0,1.0
2136851,5213550,0,4,4.0,1.0
2428365,6544132,0,4,4.0,1.0
2557142,6261447,0,4,4.0,1.0
2678167,5852578,0,4,4.0,1.0
2678180,5852580,0,4,4.0,1.0
3876792,6368817,0,4,4.0,1.0
3900938,5784915,0,4,4.0,1.0


In [17]:
bureau_balance_df.STATUS.unique()

array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object)

* C - emprestimo liquidado (closed)
* 0 - pagamento dentro do prazo
* 1, 2, 3, 4, 5 - pagamento atrasado
* X - não tem informação

In [4]:
bb_processed_df = pd.read_csv('dataset/bureau_balance_processed.csv')

In [5]:
bb_processed_df.head()

,SK_ID_BUREAU,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,SK_ID_CURR,TARGET,TRAIN
0,5001709,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,0.0
1,5001710,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-2.0,0.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,162368.0,NaN,0.0
2,5001711,-2.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,162368.0,NaN,0.0
3,5001712,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,162368.0,NaN,0.0
4,5001713,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,150635.0,NaN,0.0


In [9]:
group = bb_processed_df.groupby(by='SK_ID_CURR')['SK_ID_BUREAU'].count().reset_index()
group.head()

,SK_ID_CURR,SK_ID_BUREAU
0,100001.0,7
1,100002.0,8
2,100005.0,3
3,100010.0,2
4,100013.0,4


In [11]:
bb_processed_df[bb_processed_df.SK_ID_CURR == 100001]

,SK_ID_BUREAU,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,SK_ID_CURR,TARGET,TRAIN
297263,5896630,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-2.0,-2.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297264,5896631,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297265,5896632,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297266,5896633,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297267,5896634,1.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,-2.0,-2.0,0.0,0.0,-2.0,0.0,-2.0,0.0,0.0,-2.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297268,5896635,0.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0
297269,5896636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,100001.0,NaN,0.0


In [22]:
bureau_full_df[bureau_full_df.SK_ID_BUREAU == 5715448]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
768,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0


In [28]:
bureau_balance_df.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [102]:
tmp_df.head()

,SK_ID_BUREAU,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
0,5715448,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5715449,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5715451,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5715452,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5715453,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = pd.read_csv('dataset/bureau_balance_processed.csv')

In [4]:
df.shape

(817395, 100)

In [14]:
df.shape

(523515, 100)

In [22]:
df.head()

,SK_ID_BUREAU,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,SK_ID_CURR,TARGET
0,5008804,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,163686.0,0.0
1,5008805,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,163686.0,0.0
2,5008806,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,-2.0,0.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,0.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,356203.0,0.0
3,5008807,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,356203.0,0.0
4,5008808,0.0,-2.0,0.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,260295.0,0.0


In [12]:
df.dropna(subset = ['SK_ID_CURR', 'TARGET']).shape

(523515, 100)

In [6]:
bureau_df = pd.read_csv('dataset/bureau.csv')

In [7]:
bureau_df

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.000,0.000,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.000,171342.000,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.500,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.000,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.500,0,2700000.000,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.000,0,180000.000,71017.380,108982.620,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.000,0,42103.800,42103.800,0.000,0.0,Consumer credit,-22,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.000,0,76878.450,0.000,0.000,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.000,0,103007.700,0.000,0.000,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.000,0,4500.000,0.000,0.000,0.0,Credit card,-690,NaN
